In [1]:
import torch
import math
import torch.nn as nn
from torch.nn import functional as F
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from decode_patterns.create_images import create_images, crop_data, train_test
import numpy as np

In [2]:
batch_size = 32
num_epochs = 500
num_epochs = int(num_epochs)

In [3]:
# load data
limit = 9600
drum, bass, tempo = create_images(file_name="../patterns_pairs.tsv", limit=limit)
# drop conditionining

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function 1: 128 * 14 (+16 with cond)= 1792 --> 2048
        # веса накидываются тут
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # решение по весам
#         nn.init.normal_(self.fc1.weight, mean=0, std=1)
#         self.fc1.weight = nn.Parameter(self.fc1.weight * math.sqrt(input_dim/2))
        self.relu1 = nn.ReLU()
        
        # Linear function 2: 2048 --> 2048
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
#         # Non-linearity 2
#         nn.init.normal_(self.fc2.weight, mean=0, std=1)
#         self.fc2.weight = nn.Parameter(self.fc2.weight * math.sqrt(hidden_dim/2))
        self.relu2 = nn.ReLU()

        # Linear function 3: 2048 --> 2048
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        nn.init.normal_(self.fc3.weight, mean=0, std=1)
        self.fc3.weight = nn.Parameter(self.fc3.weight * math.sqrt(hidden_dim/2))
        self.relu3 = nn.ReLU()

        # Linear function 4 (readout): 2048 --> 128 * 36 = 4608
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
        nn.init.normal_(self.fc4.weight, mean=0, std=1)
        self.fc4.weight = nn.Parameter(self.fc4.weight * math.sqrt(hidden_dim/2))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Linear function 1
        out = self.fc1(x)
        # Non-linearity 1
        out = self.relu1(out)

        # Linear function 2
        out = self.fc2(out)
        # Non-linearity 2
        out = self.relu2(out)

        # Linear function 2
        out = self.fc3(out)
        # Non-linearity 2
        out = self.relu3(out)

        # Linear function 4 (readout)
        out = self.fc4(out)
        out = self.sigmoid(out)
        return out

In [ ]:
input_dim = 128 * 14 + 16
output_dim = 128 * 36
hidden_dim = 2048
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)


#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model.to(device)